# State Transition Model Training: All 4 Variants

This notebook trains all 4 ST model variants for burn/sham wound healing:

| Model | LoRA | mHC | Velocity | Training Time | Use Case |
|-------|------|-----|----------|---------------|----------|
| **ST-LoRA** | ✅ | ❌ | ❌ | 2-3h | Baseline parameter efficiency |
| **ST-LoRA-mHC** | ✅ | ✅ | ❌ | 3-4h | + Gradient stabilization |
| **ST-LoRA-Velocity** | ✅ | ❌ | ✅ | 3-4h | + Physics constraint |
| **ST-LoRA-mHC-Velocity** | ✅ | ✅ | ✅ | 4-5h | **Full approach** 🌟 |

**Input**: SE-600M embeddings (2000-dim, truncated from 2058)
**Task**: Predict cellular state changes from Sham → Burn conditions
**Baseline**: ST-Tahoe pretrained model (for comparison)

## 1. Environment Setup

In [ ]:
import sys
import os
from pathlib import Path
import subprocess
import time

import torch
import anndata as ad
import scanpy as sc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Add project root to path
project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root))
os.chdir(project_root)

# Check GPU availability
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.dpi'] = 100

## 2. Data Verification

In [ ]:
# Verify data files exist
data_path = "experiments/baseline_analysis/data/burn_sham_baseline_embedded_2000.h5ad"
toml_path = "experiments/st_fine_tuning/configs/burn_sham_st_training.toml"

print("Checking required files...")
print(f"  Data: {data_path} - {'✅' if os.path.exists(data_path) else '❌'}")
print(f"  Config: {toml_path} - {'✅' if os.path.exists(toml_path) else '❌'}")

if os.path.exists(data_path):
    adata = ad.read_h5ad(data_path)
    print(f"\n✓ Loaded AnnData:")
    print(f"  Shape: {adata.shape[0]} cells x {adata.shape[1]} genes")
    print(f"  Embeddings: X_state_2000 = {adata.obsm['X_state_2000'].shape}")
    print(f"  Conditions: {list(adata.obs['condition'].unique())}")
    print(f"  Timepoints: {list(adata.obs['timepoint'].unique())}")
    print(f"  Cell types: {adata.obs['cell_types_simple_short'].nunique()} types")
else:
    print("\n❌ Data file not found! Please run baseline analysis first.")

## 3. Baseline: ST-Tahoe Predictions

Load pretrained ST-Tahoe predictions (already computed).

In [ ]:
baseline_pred_path = "experiments/st_fine_tuning/results/burn_sham_st_tahoe_predictions.h5ad"

if os.path.exists(baseline_pred_path):
    adata_baseline = ad.read_h5ad(baseline_pred_path)
    print("✅ ST-Tahoe baseline predictions loaded")
    print(f"   Shape: {adata_baseline.shape}")
    print(f"   Predictions: {adata_baseline.obsm['X_state_2000'].shape}")
else:
    print("⚠️  ST-Tahoe predictions not found")
    print("   Note: ST-Tahoe trained on drugs, not suitable for burn/sham")
    print("   This serves as negative control baseline only")

## 4. Training Scripts for All 4 Variants

Create training shell scripts for each variant.

In [ ]:
# Base configuration shared across all models
base_config = {
    'toml_config_path': 'experiments/st_fine_tuning/configs/burn_sham_st_training.toml',
    'embed_key': 'X_state_2000',
    'pert_col': 'condition',
    'control_pert': 'Sham',
    'cell_type_key': 'cell_types_simple_short',
    'batch_col': 'mouse_id',
    'num_workers': 0,
    'max_steps': 10000,
    'lr': '5e-5',
    'batch_size': 8,
    'devices': 1,
    'strategy': 'auto',
}

# Define 4 model variants
model_variants = [
    {
        'name': 'st_lora',
        'display_name': 'ST-LoRA',
        'lora': True,
        'mhc': False,
        'velocity': False,
    },
    {
        'name': 'st_lora_mhc',
        'display_name': 'ST-LoRA-mHC',
        'lora': True,
        'mhc': True,
        'velocity': False,
    },
    {
        'name': 'st_lora_velocity',
        'display_name': 'ST-LoRA-Velocity',
        'lora': True,
        'mhc': False,
        'velocity': True,
    },
    {
        'name': 'st_lora_mhc_velocity',
        'display_name': 'ST-LoRA-mHC-Velocity',
        'lora': True,
        'mhc': True,
        'velocity': True,
    },
]

print("Model Variants:")
print("=" * 80)
for i, variant in enumerate(model_variants, 1):
    print(f"{i}. {variant['display_name']}")
    print(f"   - LoRA: {'✅' if variant['lora'] else '❌'}")
    print(f"   - mHC: {'✅' if variant['mhc'] else '❌'}")
    print(f"   - Velocity: {'✅' if variant['velocity'] else '❌'}")
    print()

## 5. Generate Training Scripts

In [ ]:
def generate_training_script(variant, base_config):
    """Generate training script for a model variant."""
    
    script_lines = [
        "#!/bin/bash",
        "",
        f"# Training script for {variant['display_name']}",
        f"# Generated: {time.strftime('%Y-%m-%d %H:%M:%S')}",
        "",
        "cd /home/scumpia-mrl/Desktop/Sujit/Projects/state-experimentation",
        "",
        "state tx train \\",
        f"  data.kwargs.toml_config_path={base_config['toml_config_path']} \\",
        f"  data.kwargs.embed_key={base_config['embed_key']} \\",
        f"  data.kwargs.pert_col={base_config['pert_col']} \\",
        f"  data.kwargs.control_pert={base_config['control_pert']} \\",
        f"  data.kwargs.cell_type_key={base_config['cell_type_key']} \\",
        f"  data.kwargs.batch_col={base_config['batch_col']} \\",
        f"  data.kwargs.num_workers={base_config['num_workers']} \\",
    ]
    
    # LoRA config
    if variant['lora']:
        script_lines.extend([
            "  +model.kwargs.lora.enable=true \\",
            "  +model.kwargs.lora.r=16 \\",
            "  +model.kwargs.lora.alpha=32 \\",
        ])
    
    # mHC config
    if variant['mhc']:
        script_lines.extend([
            "  +model.kwargs.use_mhc=true \\",
            "  +model.kwargs.mhc.sinkhorn_iters=10 \\",
        ])
    else:
        script_lines.append("  +model.kwargs.use_mhc=false \\")
    
    # Velocity config
    if variant['velocity']:
        script_lines.extend([
            "  +model.kwargs.use_velocity_alignment=true \\",
            "  +model.kwargs.velocity_lambda=0.1 \\",
            "  +model.kwargs.velocity_beta=1.0 \\",
            "  +model.kwargs.velocity_warmup_steps=1000 \\",
        ])
    else:
        script_lines.append("  +model.kwargs.use_velocity_alignment=false \\")
    
    # Training config
    script_lines.extend([
        f"  training.max_steps={base_config['max_steps']} \\",
        f"  training.lr={base_config['lr']} \\",
        f"  training.batch_size={base_config['batch_size']} \\",
        f"  training.devices={base_config['devices']} \\",
        f"  training.strategy={base_config['strategy']} \\",
        f"  output_dir=/home/scumpia-mrl/state_models/{variant['name']} \\",
        f"  name={variant['name']}_burn_sham",
    ])
    
    return "\n".join(script_lines)

# Generate scripts
scripts_dir = Path("experiments/st_fine_tuning/train_scripts")
scripts_dir.mkdir(parents=True, exist_ok=True)

for variant in model_variants:
    script_content = generate_training_script(variant, base_config)
    script_path = scripts_dir / f"train_{variant['name']}.sh"
    
    with open(script_path, 'w') as f:
        f.write(script_content)
    
    # Make executable
    os.chmod(script_path, 0o755)
    
    print(f"✅ Created: {script_path}")

print("\n✓ All training scripts generated!")

## 6. Train All Models (Sequential)

**Note**: Training takes 2-5 hours per model. Run overnight or on compute cluster.

In [ ]:
# Option 1: Train all models sequentially (uncomment to run)
# WARNING: This will take 10-15 hours total!

def train_model(variant_name):
    """Train a single model variant."""
    script_path = f"experiments/st_fine_tuning/train_scripts/train_{variant_name}.sh"
    log_path = f"experiments/st_fine_tuning/logs/{variant_name}_training.log"
    
    print(f"\n{'='*80}")
    print(f"Training: {variant_name}")
    print(f"Script: {script_path}")
    print(f"Log: {log_path}")
    print(f"{'='*80}")
    
    # Create logs directory
    Path("experiments/st_fine_tuning/logs").mkdir(parents=True, exist_ok=True)
    
    # Run training
    with open(log_path, 'w') as log_file:
        process = subprocess.Popen(
            ['bash', script_path],
            stdout=log_file,
            stderr=subprocess.STDOUT,
        )
        
        print(f"Training started (PID: {process.pid})")
        print(f"Monitor with: tail -f {log_path}")
        
        # Wait for completion
        process.wait()
        
        if process.returncode == 0:
            print(f"✅ Training completed successfully")
        else:
            print(f"❌ Training failed with exit code {process.returncode}")
            return False
    
    return True

# Uncomment to train all models
# for variant in model_variants:
#     success = train_model(variant['name'])
#     if not success:
#         print(f"\n❌ Training failed for {variant['name']}, stopping.")
#         break

print("\n📝 To train models, uncomment the code above or run scripts manually:")
for variant in model_variants:
    print(f"  bash experiments/st_fine_tuning/train_scripts/train_{variant['name']}.sh")

## 7. Monitor Training Progress

In [ ]:
# Check which models have been trained
print("Training Status:")
print("=" * 80)

for variant in model_variants:
    model_dir = f"/home/scumpia-mrl/state_models/{variant['name']}"
    log_path = f"experiments/st_fine_tuning/logs/{variant['name']}_training.log"
    
    status = "⏳ Not started"
    if os.path.exists(log_path):
        status = "🔄 In progress or completed"
        if os.path.exists(f"{model_dir}/checkpoints/best.ckpt"):
            status = "✅ Completed"
    
    print(f"{variant['display_name']:30s} {status}")
    
    if os.path.exists(log_path):
        print(f"  Log: {log_path}")
        print(f"  TensorBoard: tensorboard --logdir={model_dir}")
    print()

## 8. Training Commands Summary

Copy-paste ready commands for manual execution.

In [ ]:
print("Training Commands:")
print("=" * 80)
print("\n# Train all models sequentially:")
for variant in model_variants:
    print(f"\n# {variant['display_name']}")
    print(f"bash experiments/st_fine_tuning/train_scripts/train_{variant['name']}.sh 2>&1 | tee experiments/st_fine_tuning/logs/{variant['name']}_training.log")

print("\n" + "=" * 80)
print("\n# Monitor training:")
for variant in model_variants:
    print(f"\n# {variant['display_name']}")
    print(f"tail -f experiments/st_fine_tuning/logs/{variant['name']}_training.log")
    print(f"tensorboard --logdir=/home/scumpia-mrl/state_models/{variant['name']}")

## 9. Next Steps

After training completes:
1. Run predictions with each trained model
2. Use `compare_st_results.ipynb` for comprehensive evaluation
3. Compare training dynamics (loss curves, gradient stability)
4. Evaluate perturbation prediction accuracy
5. Biological validation (wound healing trajectories)

In [ ]:
print("Next Steps:")
print("=" * 80)
print("\n1. Wait for training to complete (10-15 hours total)")
print("2. Run inference for each model (see compare_st_results.ipynb)")
print("3. Compare results across all variants")
print("4. Document findings in paper/presentation")
print("\n✅ All training scripts ready!")